![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FRetrieval&file=Retrieval+-+Memorystore.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Memorystore.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FRetrieval%2FRetrieval%2520-%2520Memorystore.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Memorystore.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Memorystore.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Retrieval - Memorystore (Redis)
<p style="font-size: 45px;">IN PROGRESS - NOT COMPLETE</p>

In prior workflows, a series of documents was [processed into chunks](../Chunking/readme.md), and for each chunk, [embeddings](../Embeddings/readme.md) were created:

- Process: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb)
- Embed: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

Retrieving chunks for a query involves calculating the embedding for the query and then using similarity metrics to find relevant chunks. A thorough review of similarity matching can be found in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb) - use dot product! As development moves from experiment to application, the process of storing and computing similarity is migrated to a [retrieval](./readme.md) system. This workflow is part of a [series of workflows exploring many retrieval systems](./readme.md).

**Memorystore For Storage, Indexing, And Search**

[Google Cloud Memorystore](https://cloud.google.com/memorystore) is a fully managed in-memory data store service that can be used to enhance the performance of applications by caching frequently accessed data. Memorystore offers several in-memory database options, including [Redis](https://cloud.google.com/memorystore/docs/redis), [Redis Cluster](https://cloud.google.com/memorystore/docs/cluster), [Memcached](https://cloud.google.com/memorystore/docs/memcached), and [Valkey](https://cloud.google.com/memorystore/docs/valkey/). Each option provides unique features and capabilities to suit different needs.

This example uses [Redis](https://redis.io/docs/latest/), which is an open-source, in-memory data store that can be used as a database, cache, message broker, and streaming engine. In Redis, data is stored as key-value pairs. The key serves as an identifier to request or retrieve specific data. Redis offers a [variety of data types](https://redis.io/docs/latest/develop/data-types/) for values, including strings, lists, sets, hashes, and sorted sets. These data types provide flexibility in how you structure and organize your data. In addition to basic retrieval using keys, Redis offers querying capabilities across data elements, [including vector search](https://cloud.google.com/memorystore/docs/redis/about-vector-search) for efficient similarity matching.

**Use Case Data**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:

- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging.

**Approaches**

[This series](../readme.md) covers many generative AI workflows. These documents are used directly as long context for Gemini in the workflow [Long Context Retrieval With The Vertex AI Gemini API](../Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb). The workflow below uses a [retrieval](./readme.md) approach with the already generated chunks and embeddings.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.redis', 'google-cloud-redis'),
    ('redis', 'redis', '5.0.8')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable redis.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'retrieval-memorystore-redis'

# Redis names
REDIS_INSTANCE_NAME = EXPERIMENT

Packages

In [8]:
import os, json, time, glob, datetime, tempfile

import numpy as np
import redis

# Vertex AI
from google.cloud import aiplatform
import vertexai.language_models # for embeddings API
import vertexai.generative_models # for Gemini Models
from vertexai.resources.preview import feature_store

# memorystore
from google.cloud import redis_v1

In [9]:
aiplatform.__version__

'1.69.0'

Clients

In [10]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

# memorystore clients
redis_client = redis_v1.CloudRedisClient()

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an example of processing mulitple large pdfs (over 1000 pages) into chunks: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [11]:
local_dir = '../Embeddings/files/embeddings-api'

In [12]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [13]:
jsonl_files = glob.glob(f"{local_dir}/large-files*.jsonl")
jsonl_files.sort()
jsonl_files

['../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0000.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0001.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0002.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0003.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0004.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0005.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0006.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0007.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0008.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0009.jsonl']

In [14]:
chunks = []
for file in jsonl_files:
    with open(file, 'r') as f:
        chunks.extend([json.loads(line) for line in f])
len(chunks)

9040

### Review A Chunk

In [15]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [16]:
chunks[0]['instance']['chunk_id']

'fannie_part_0_c17'

In [17]:
print(chunks[0]['instance']['content'])

# Selling Guide Fannie Mae Single Family

## Fannie Mae Copyright Notice

### Fannie Mae Copyright Notice

|-|
| Section B3-4.2, Verification of Depository Assets 402 |
| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |
| B3-4.2-02, Depository Accounts (12/14/2022) 405 |
| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |
| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |
| B3-4.2-05, Foreign Assets (05/04/2022) 411 |
| Section B3-4.3, Verification of Non-Depository Assets 412 |
| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |
| B3-4.3-02, Trust Accounts (04/01/2009) 413 |
| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |
| B3-4.3-04, Personal Gifts (09/06/2023) 415 |
| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |
| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |
| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |
| B3-4.3-08, Employer Assistance (09/29/2015) 423 |
| B3-4.3-09,

In [18]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.031277116388082504,
 0.03056905046105385,
 0.010865348391234875,
 0.0623614676296711,
 0.03228681534528732,
 0.05066155269742012,
 0.046544693410396576,
 0.05509665608406067,
 -0.014074751175940037,
 0.008380400016903877]

### Prepare Chunk Structure

Make a list of dictionaries with information for each chunk:

In [40]:
content_chunks = [
    dict(
        gse = chunk['instance']['gse'],
        chunk_id = chunk['instance']['chunk_id'],
        content = chunk['instance']['content'],
        embedding = chunk['predictions'][0]['embeddings']['values']
    ) for chunk in chunks
]

### Query Embedding

Create a query, or prompt, and get the embedding for it:

Connect to models for text embeddings. Learn more about the model API:
- [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

In [20]:
question = "Does a lender have to perform servicing functions directly?"

In [21]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')

In [22]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.0005117303808219731,
 0.009651427157223225,
 0.01768726110458374,
 0.014538003131747246,
 -0.01829824410378933,
 0.027877431362867355,
 -0.021124685183167458,
 0.008830446749925613,
 -0.02669006586074829,
 0.06414774805307388]

---
## Retrieval With Memorystore (Redis)

Memorystore offers multiple caching engines as a service: [Valkey](https://cloud.google.com/memorystore/docs/valkey/), [Redis](https://cloud.google.com/memorystore/docs/redis), [Redis Cluster](https://cloud.google.com/memorystore/docs/cluster), and [Memcached](https://cloud.google.com/memorystore/docs/memcached). Redis is offered as a single instance or a clustered instance where the instance is a series of shards that contains subsets of the cached data.

This workflow will use a single instance of [Redis](https://cloud.google.com/memorystore/docs/redis) because it [supports vector search](https://cloud.google.com/memorystore/docs/redis/about-vector-search).

**Memorystore data structure**

Memorystore, Redis in this case, is a key:value pair database.  In this case think of the key as the value that would be used to requests or retrieve data for.  And data is actually a flexible, multi-parameter, object with multiple [possible data types](https://redis.io/docs/latest/develop/data-types/).  Beyond retriving data based on the key there are also querying capabilties across the data elements, including vector search!

**Understanding Costs**

[Pricing for Memorystore for Redis](https://cloud.google.com/memorystore/docs/redis/pricing) is straightforward and based on these parameters that are part of the instance creation:
- **Service Tier**: [read more about service tiers](https://cloud.google.com/memorystore/docs/redis/redis-tiers)
    - Basic - a simple Redis cache with a standalone instance (**Used in this workflow.**)
    - Standard - A high availability instance with cross-zone replication and automatic failover
- **Provisioned capacity**: 
    - Size in GB priced in $ per GB per hour. Capacity tiers make additional capacity progressively cheaper. (**The Minimum of 1 GB is used in this workflow.**)
- **Region**: 
    - Pricing varies by the region choosen for the instance.
- **Replicas**: 
    - For the Standard Service Tier you have the option to enable read replicas for distributed reads.
    
    
Important Notes on the Approach used here:

Redis has a concept called modules: https://redis.io/docs/latest/develop/reference/modules/

Redis has core data types that are directly supported: https://redis.io/docs/latest/develop/data-types/#core-data-types

Additional data types are enabled by 

The data types supported in Redis are expanded extension modules: https://redis.io/docs/latest/develop/data-types/#extension-data-types

Of note is the JSON data type module extension: https://redis.io/docs/latest/develop/data-types/json/

Google Cloud Memorystore Redis support these versions of Redis: https://cloud.google.com/memorystore/docs/redis/supported-versions

Which inclues version 7.2 and the build in vector search functionality: https://cloud.google.com/memorystore/docs/redis/supported-versions#redis_version_72

Many examples found on the web for using this vector search capability use it with the JSON module, like this one: https://redis.io/docs/latest/develop/get-started/vector-database/

An example directly using hashing can be found here: https://redis-py.readthedocs.io/en/stable/examples/search_vector_similarity_examples.html

In the supported functionality for Memorystore Redis, modules are not supported.  For this reason, the HASH data type is best to use for vector search feature.  While a Python dictionary is comparable to a HASH
- https://redis.io/docs/latest/develop/data-types/#hashes
    - https://redis.io/docs/latest/develop/data-types/hashes/
- Trying to store a dictionary element that has a value of type list is not possilbe. Like an embedding as a list of floats.  Serializing the embedding as bytes is necessary and covered by this note:
    - https://cloud.google.com/memorystore/docs/redis/indexing-vectors



Redis Hashes:

https://redis.io/docs/latest/develop/data-types/hashes/

Can store more key-value pairs than you probably need or have available memory for: (2^32-1).

Offer many features like field expiration.

More on redis:
database, namespaces(prefixes), keys


### Create/Retrieve An Instance

https://cloud.google.com/memorystore/docs/redis/redis-tiers

https://cloud.google.com/python/docs/reference/redis/latest/google.cloud.redis_v1.services.cloud_redis.CloudRedisClient

In [27]:
try:
    redis_instance = redis_client.get_instance(
        name = f"projects/{PROJECT_ID}/locations/{REGION}/instances/{REDIS_INSTANCE_NAME}"
    )
    print(f"Retrieved Redis instance: {redis_instance.name}")
except Exception:
    print(f"Creating Redis instance ...")
    create_instance = redis_client.create_instance(
        parent = f"projects/{PROJECT_ID}/locations/{REGION}",
        instance_id = REDIS_INSTANCE_NAME,
        instance = redis_v1.Instance(
            name = f"projects/{PROJECT_ID}/locations/{REGION}/instances/{REDIS_INSTANCE_NAME}",
            tier = redis_v1.Instance.Tier.BASIC,
            memory_size_gb = 1,
            redis_version = 'REDIS_7_2',
            transit_encryption_mode=redis_v1.Instance.TransitEncryptionMode.SERVER_AUTHENTICATION  # Enable TLS
        )
    )
    response = create_instance.result()
    redis_instance = redis_client.get_instance(
        name = f"projects/{PROJECT_ID}/locations/{REGION}/instances/{REDIS_INSTANCE_NAME}"
    )
    print(f"Created Redis instance: {redis_instance.name}")

Creating Redis instance ...
Created Redis instance: projects/statmike-mlops-349915/locations/us-central1/instances/retrieval-memorystore-redis


In [28]:
redis_instance

name: "projects/statmike-mlops-349915/locations/us-central1/instances/retrieval-memorystore-redis"
location_id: "us-central1-c"
redis_version: "REDIS_7_2"
reserved_ip_range: "10.106.20.32/29"
host: "10.106.20.35"
port: 6378
current_location_id: "us-central1-c"
create_time {
  seconds: 1730169733
  nanos: 767497286
}
state: READY
tier: BASIC
memory_size_gb: 1
authorized_network: "projects/statmike-mlops-349915/global/networks/default"
persistence_iam_identity: "serviceAccount:service-1026793852137@cloud-redis.iam.gserviceaccount.com"
connect_mode: DIRECT_PEERING
server_ca_certs {
  serial_number: "0"
  cert: "-----BEGIN CERTIFICATE-----\nMIIDnTCCAoWgAwIBAgIBADANBgkqhkiG9w0BAQsFADCBhTEtMCsGA1UELhMkOTVm\nZmNlYmUtMDAwZi00NzYyLTkwYzctOGYxZWZhZGI1NzkxMTEwLwYDVQQDEyhHb29n\nbGUgQ2xvdWQgTWVtb3J5c3RvcmUgUmVkaXMgU2VydmVyIENBMRQwEgYDVQQKEwtH\nb29nbGUsIEluYzELMAkGA1UEBhMCVVMwHhcNMjQxMDI5MDI0MTIxWhcNMzQxMDI3\nMDI0MjIxWjCBhTEtMCsGA1UELhMkOTVmZmNlYmUtMDAwZi00NzYyLTkwYzctOGYx\nZWZhZGI1NzkxMTEwLwYDVQQDE

### Connect To Redis Instance

Connect to the Redis instance using the [redis-py](https://redis-py.readthedocs.io/en/stable/) Python package. The following code create two clients using [redis.Redis()](https://redis-py.readthedocs.io/en/stable/connections.html#generic-client):
- `r_client` with `decode_responses = True`
- `b_client` with `decode_responses = False`

Why two clients?  We will be storing data in the [Redis Hashes](https://redis.io/docs/latest/develop/data-types/hashes/) data type which is like a Python dictionary. Further, the embedding vectors will be converted to bytes objects before writing to Redis.  

## more here
- why bytes for the vectors
- links to GCP info on this
- two methods of using client: native and .execute_command
- links to redis-py and GCP docs for langauge

In [30]:
with tempfile.NamedTemporaryFile(delete=False) as cert_file:
    cert_file.write(redis_instance.server_ca_certs[0].cert.encode())  # Write certificate content
    cert_path = cert_file.name

In [31]:
r_client = redis.Redis(
    host = redis_instance.host,
    port = redis_instance.port,
    ssl = True,
    ssl_ca_certs = cert_path,
    decode_responses = True
)

In [32]:
b_client = redis.Redis(
    host = redis_instance.host,
    port = redis_instance.port,
    ssl = True,
    ssl_ca_certs = cert_path,
    decode_responses = False
)

In [33]:
try:
    r_client.ping() and b_client.ping()
    print("Connected to Redis successfully!")
except Exception as e:
    print(f"Error connecting to Redis: {e}")

Connected to Redis successfully!


### Prepare Data For Redis

There are multiple storage [data formats](https://redis.io/docs/latest/develop/data-types/) possible with Redis.  [Hashing](https://redis.io/docs/latest/develop/data-types/hashes/) is like a Python dictionary and offers many advantages, like the vector search capability that will be used here.  To use the hash data type the embedding need to be serialized before inserting: https://cloud.google.com/memorystore/docs/redis/indexing-vectors

In [41]:
content_chunks[0].keys()

dict_keys(['gse', 'chunk_id', 'content', 'embedding'])

In [42]:
for chunk in content_chunks:
    chunk['embedding'] = np.array(chunk['embedding']).astype('float32').tobytes()

In [43]:
type(content_chunks[0]['embedding'])

bytes

In [44]:
len(content_chunks[0]['embedding'])

3072

### Add/Retrive/Delete Records To The Instance

In [45]:
first_record = content_chunks[0]

In [46]:
first_record.keys()

dict_keys(['gse', 'chunk_id', 'content', 'embedding'])

In [47]:
first_record['chunk_id']

'fannie_part_0_c17'

In [50]:
if r_client.exists(first_record['chunk_id']):
    print(f"Found this record already in the database: {first_record['chunk_id']}")
else:
    print(f"Adding the record to the database: {first_record['chunk_id']}")
    r_client.hset(first_record['chunk_id'], mapping = first_record)

Adding the record to the database: fannie_part_0_c17


In [51]:
r_client.hget(first_record['chunk_id'], 'gse')

'fannie'

In [54]:
b_client.hget(first_record['chunk_id'], 'gse')

b'fannie'

In [52]:
r_client.hget(first_record['chunk_id'], 'content')

'# Selling Guide Fannie Mae Single Family\n\n## Fannie Mae Copyright Notice\n\n### Fannie Mae Copyright Notice\n\n|-|\n| Section B3-4.2, Verification of Depository Assets 402 |\n| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |\n| B3-4.2-02, Depository Accounts (12/14/2022) 405 |\n| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |\n| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |\n| B3-4.2-05, Foreign Assets (05/04/2022) 411 |\n| Section B3-4.3, Verification of Non-Depository Assets 412 |\n| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |\n| B3-4.3-02, Trust Accounts (04/01/2009) 413 |\n| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |\n| B3-4.3-04, Personal Gifts (09/06/2023) 415 |\n| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |\n| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |\n| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |\n| B3-4.3-08, Employer Assistance (09/29/20

In [53]:
r_client.hmget(first_record['chunk_id'], ['gse', 'content'])

['fannie',
 '# Selling Guide Fannie Mae Single Family\n\n## Fannie Mae Copyright Notice\n\n### Fannie Mae Copyright Notice\n\n|-|\n| Section B3-4.2, Verification of Depository Assets 402 |\n| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |\n| B3-4.2-02, Depository Accounts (12/14/2022) 405 |\n| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |\n| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |\n| B3-4.2-05, Foreign Assets (05/04/2022) 411 |\n| Section B3-4.3, Verification of Non-Depository Assets 412 |\n| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |\n| B3-4.3-02, Trust Accounts (04/01/2009) 413 |\n| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |\n| B3-4.3-04, Personal Gifts (09/06/2023) 415 |\n| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |\n| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |\n| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |\n| B3-4.3-08, Employer Assistan

In [55]:
r_client.hdel(first_record['chunk_id'], 'embedding')

1

In [56]:
r_client.hgetall(first_record['chunk_id'])

{'chunk_id': 'fannie_part_0_c17',
 'content': '# Selling Guide Fannie Mae Single Family\n\n## Fannie Mae Copyright Notice\n\n### Fannie Mae Copyright Notice\n\n|-|\n| Section B3-4.2, Verification of Depository Assets 402 |\n| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |\n| B3-4.2-02, Depository Accounts (12/14/2022) 405 |\n| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |\n| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |\n| B3-4.2-05, Foreign Assets (05/04/2022) 411 |\n| Section B3-4.3, Verification of Non-Depository Assets 412 |\n| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |\n| B3-4.3-02, Trust Accounts (04/01/2009) 413 |\n| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |\n| B3-4.3-04, Personal Gifts (09/06/2023) 415 |\n| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |\n| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |\n| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423

In [57]:
r_client.delete(first_record['chunk_id'])

1

### Load All Documents To The Instance

Use a pipeline to efficiently load all the records.

https://redis-py.readthedocs.io/en/stable/examples/pipeline_examples.html

In [58]:
# check for chunk loading status
with r_client.pipeline() as pipe:
    for chunk in content_chunks:
        pipe.exists(chunk['chunk_id'])
    exists_results = pipe.execute()

In [59]:
# load missing chunks
with r_client.pipeline() as pipe:
    load_indexes = []
    for i, (chunk, exists) in enumerate(zip(content_chunks, exists_results)):
        if not exists:
            load_indexes.append(i)
            pipe.hset(chunk['chunk_id'], mapping=chunk)
    load_results = pipe.execute()

In [60]:
# check for loading issues and give first failure id info for diagnostic
if all(load_results):
    print(f'All chunks({len(load_results)}) loaded successfully.')
else:
    print(f"During loading {load_results.count(0)} records were not successfully loaded.")
    first_fail_index = load_indexes[load_results.index(0)]
    print(f"Start troubleshooting with the record at index {first_fail_index} which as has 'chunk_id' = {content_chunks[first_fail_index]['chunk_id']}")

All chunks(9040) loaded successfully.


In [61]:
r_client.hget(first_record['chunk_id'], 'gse')

'fannie'

In [62]:
r_client.hlen(first_record['chunk_id'])

4

In [63]:
r_client.dbsize()

9040

### Create An Index

https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/vectors/

https://redis-py.readthedocs.io/en/stable/examples/search_vector_similarity_examples.html#KNN-Queries

https://redis.io/docs/latest/develop/get-started/vector-database/#create-an-index

In [173]:
INDEX_NAME = 'flat-index'

try:
    check_index = client.ft(INDEX_NAME).info()
    print(f"This index already exists: {check_index['index_name']}")
except Exception:
    print(f'Create the index ...')
    schema = (
        #redis.commands.search.field.TagField(name = "gse"),
        redis.commands.search.field.VectorField(
            'embedding',
            'FLAT',
            dict(
                TYPE = 'FLOAT32',
                DIM = len(question_embedding),
                DISTANCE_METRIC = 'IP' # dot product
            )
        )
    )
    index_def = redis.commands.search.indexDefinition.IndexDefinition(
        #prefix = [], # defaults to all, this is a list of namespaces/prefixes to index
        index_type = redis.commands.search.indexDefinition.IndexType.HASH
    )
    client.ft(INDEX_NAME).create_index(fields = schema, definition = index_def)
    check_index = client.ft(INDEX_NAME).info()
    print(f"Index Created: {check_index['index_name']}")

Create the index ...
Index Created: flat-index


In [174]:
INDEX_NAME = 'hnsw-index'

try:
    check_index = client.ft(INDEX_NAME).info()
    print(f"This index already exists: {check_index['index_name']}")
except Exception:
    print(f'Create the index ...')
    schema = (
        #redis.commands.search.field.TagField(name = "gse"),
        redis.commands.search.field.VectorField(
            'embedding',
            'HNSW',
            dict(
                TYPE = 'FLOAT32',
                DIM = len(question_embedding),
                DISTANCE_METRIC = 'IP' # dot product
            )
        )
    )
    index_def = redis.commands.search.indexDefinition.IndexDefinition(
        #prefix = [], # defaults to all, this is a list of namespaces/prefixes to index
        index_type = redis.commands.search.indexDefinition.IndexType.HASH
    )
    client.ft(INDEX_NAME).create_index(fields = schema, definition = index_def)
    check_index = client.ft(INDEX_NAME).info()
    print(f"Index Created: {check_index['index_name']}")

Create the index ...
Index Created: hnsw-index


In [ ]:
INDEX_NAME = 'flat-index'

command = (
    f"FT.CREATE {INDEX_NAME} ON HASH "
    f"SCHEMA embedding VECTOR FLAT 6 "
    f"TYPE FLOAT32 "
    f"DIM {len(question_embedding)} "
    f"DISTANCE_METRIC IP "
)


client.execute_command(command)

In [65]:
INDEX_NAME = 'flat-index'

try:
    check_index = r_client.ft(INDEX_NAME).info()
    print(f"This index already exists: {check_index['index_name']}")
except Exception:
    print(f'Create the index ...')

    command = (
        f"FT.CREATE {INDEX_NAME} ON HASH "
        f"SCHEMA embedding VECTOR FLAT 6 "
        f"TYPE FLOAT32 "
        f"DIM {len(question_embedding)} "
        f"DISTANCE_METRIC IP "
    )

    r_client.execute_command(command)
    
    print(f'Checking for index backfill ...')
    while True:
        check_index = r_client.ft(INDEX_NAME).info()
        if check_index['backfill_in_progress'] == '1':
            complete_pct = 100*float(check_index['backfill_complete_percent'])
            print(f"Backfill still in progress with {complete_pct:.2f} percent complete ...")
            time.sleep(10)
        else:
            print(f"Backfill complete.")
            print(f"Index created and covers {check_index['num_docs']} records")
            break

This index already exists: flat-index


In [69]:
INDEX_NAME = 'hnsw-index'

try:
    check_index = r_client.ft(INDEX_NAME).info()
    print(f"This index already exists: {check_index['index_name']}")
except Exception:
    print(f'Create the index ...')

    command = (
        f"FT.CREATE {INDEX_NAME} ON HASH "
        f"SCHEMA embedding VECTOR HNSW 6 "
        f"TYPE FLOAT32 "
        f"DIM {len(question_embedding)} "
        f"DISTANCE_METRIC IP "
    )

    r_client.execute_command(command)
    
    print(f'Checking for index backfill ...')
    while True:
        check_index = r_client.ft(INDEX_NAME).info()
        if check_index['backfill_in_progress'] == '1':
            complete_pct = 100*float(check_index['backfill_complete_percent'])
            print(f"Backfill still in progress with {complete_pct:.2f} percent complete ...")
            time.sleep(1)
        else:
            print(f"Backfill complete.")
            print(f"Index created and covers {check_index['num_docs']} records")
            break

Create the index ...
Checking for index backfill ...
Backfill still in progress with 0.00 percent complete ...
Backfill still in progress with 22.72 percent complete ...
Backfill still in progress with 39.81 percent complete ...
Backfill still in progress with 53.53 percent complete ...
Backfill still in progress with 65.74 percent complete ...
Backfill still in progress with 77.05 percent complete ...
Backfill still in progress with 87.60 percent complete ...
Backfill still in progress with 97.61 percent complete ...
Backfill complete.
Index created and covers 9040 records


### Check The Index State

In [73]:
r_client.execute_command(
    'FT._LIST'
)

['flat-index', 'hnsw-index']

In [74]:
check_index = r_client.ft('flat-index').info()

In [75]:
check_index

{'index_name': 'flat-index',
 'index_options': [],
 'index_definition': ['key_type',
  'HASH',
  'prefixes',
  [''],
  'default_score',
  '1'],
 'attributes': [['identifier',
   'embedding',
   'attribute',
   'embedding',
   'type',
   'VECTOR',
   'index',
   ['capacity',
    10240,
    'dimensions',
    768,
    'distance_metric',
    'IP',
    'data_type',
    'FLOAT32',
    'algorithm',
    ['name', 'FLAT', 'block_size', 1024]]]],
 'num_docs': '9040',
 'num_terms': '0',
 'num_records': '9040',
 'hash_indexing_failures': '0',
 'backfill_in_progress': '0',
 'backfill_complete_percent': '1.000000',
 'mutation_queue_size': '0',
 'recent_mutations_queue_delay': '0 sec'}

### Matching

In [93]:
def parse_matches(results, include_embed = False):
    encoding = b_client.get_encoder().encoding
    responses = []
    for i in range(2, len(results), 2):
        response = {}
        for j in range(0, len(results[i]), 2):
            key = results[i][j].decode(encoding)
            value = results[i][j+1]
            if key == 'embedding':
                if include_embed:
                    response[key] = np.frombuffer(value, dtype='float32').tolist()
            elif key == 'distance':
                response[key] = float(value.decode(encoding))
            else:
                response[key] = value.decode(encoding)
        responses.append(response)
    return responses

In [94]:
query_args = [
    f"FT.SEARCH {INDEX_NAME}",
    f"*=>[KNN 5 @embedding $query_embedding AS distance]",
    "PARAMS",
    2,
    "query_embedding",
    np.array(question_embedding).astype('float32').tobytes(),
    "DIALECT",
    2,
]

results = b_client.execute_command(*query_args)

In [95]:
matches = parse_matches(results)

In [96]:
matches

[{'distance': 0.331650435925,
  'chunk_id': 'fannie_part_0_c326',
  'content': "# Chapter A3-3, Third-Party Lending Functions and Servicing Arrangements\n\nIntroduction This chapter explains Fannie Mae's requirements regarding the outsourcing of mortgage origination and servicing functions. It also addresses other servicing arrangements as well as the requirements related to document custody and document custodians. 105",
  'gse': 'fannie'},
 {'distance': 0.327629327774,
  'chunk_id': 'fannie_part_0_c353',
  'content': "# A3-3-03, Other Servicing Arrangements (12/15/2015)\n\n## General Requirements for Subservicing Arrangements\n\nA servicer may use a subservicer only if it will not interfere with the servicer's ability to meet Fannie Mae's remitting and reporting requirements. A master servicer may not enter into new subservicing arrangements—or extend existing arrangements to include newly originated mortgages-unless both the master servicer and the subservicer are Fannie Mae- approv

## Remove Resources

In [252]:
#client.flushall() # empty all records in the instance, across all databases

In [26]:
#redis_client.delete_instance(name = redis_instance.name)